<a href="https://colab.research.google.com/github/llai7/MATH227C/blob/main/Math227C20Sp_P09_LuluLai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Set 9

## Example: predicting colon cancer from stool microbiome composition

In [2]:
# plot settings
options(repr.plot.width=15, repr.plot.height=10)

In [3]:
# Install a package BioConductor ExperimentHub to access the example data
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install()
BiocManager::install("ExperimentHub")

# Install glmnet for LASSO and Elastic Net regression
install.packages("glmnet")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.rstudio.com


Bioconductor version 3.15 (BiocManager 1.30.18), R 4.2.0 (2022-04-22)

Installing package(s) 'BiocVersion'

Old packages: 'openssl', 'roxygen2', 'tinytex', 'usethis', 'xfun'

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.rstudio.com


Bioconductor version 3.15 (BiocManager 1.30.18), R 4.2.0 (2022-04-22)

Installing package(s) 'ExperimentHub'

also installing the dependencies ‘bitops’, ‘zlibbioc’, ‘RCurl’, ‘GenomeInfoDbData’, ‘XVector’, ‘GenomeInfoDb’, ‘lazyeval’, ‘png’, ‘Biostrings’, ‘httpuv’, ‘xtable’, ‘fontawesome’, ‘sourcetools’, ‘later’, ‘promises’, ‘htmlwidgets’, ‘crosstalk’, ‘Rcpp’, ‘plogr’, ‘Biobase’, ‘IRanges’, ‘KEGGREST’, ‘shiny’, ‘DT’,

In [4]:

library("BiocManager")
library("glmnet")


Loading required package: Matrix

Loaded glmnet 4.1-4



In [5]:
library("ExperimentHub")

eh = ExperimentHub()
data = eh[["EH361"]]


Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: AnnotationHub

Loading required package: BiocFileCache

Loading required package: dbplyr

snapshotDate(): 2022-04-26

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.rstudio.com


Bioconductor version 3.15 (BiocManager 1.30.18), R 4.2.0 (2022-04-22)

Installing

## 1. Explore the data set

Explore our data set. The rows are the presence of microbes in the gut. They also contain the presence of cancer. The columns represent different patients. 

__i) How many patients are in the data set?__

__ii) How many species of microbe are in the data set?__

There are possible 3 disease states. Create a data set with only patients who are either "N" (No cancer) or "cancer" (cancer). In other words, remove the patiends with adenomas.

__iii) After removing the patients with adenomas, how many patients are in the data set?__

In [8]:
print(paste('There are',ncol(data),'patients.' ,sep=' '))

[1] "There are 156 patients."


In [12]:
print(paste('There are',nrow(data),'species of microbe in the data set.' ,sep=' '))

[1] "There are 1505 species of microbe in the data set."


For simplicity, let's only use the "n" and "cancer" states (and remove the adenomas).


In [14]:
dataCancerNoCancer = data[, data$disease %in% c("n", "cancer")]

print(paste('There are',ncol(dataCancerNoCancer),'patients in the data set now.' ,sep=' '))

[1] "There are 114 patients in the data set now."


## 2. Lasso regression
Let's perform lasso regression. In this case, the response variable is categorical (cancer or no cancer) so we can use a binomial model, which is a subset of logistic models. 

__i) According to the crossvalidation analysis, how many species of microbe should we include in a predictive model of colon cancer?__

In [ ]:
y = factor(dataCancerNoCancer$disease)
x = t(exprs(dataCancerNoCancer))

lassoFit = glmnet(y=y, x=x, family="binomial")

plot(lassoFit, xvar = "lambda", label = TRUE)


In [ ]:
crossValidationOutput <- cv.glmnet(y=factor(dataCancerNoCancer$disease),
                                   x=t(exprs(dataCancerNoCancer)), family="binomial")

plot(crossValidationOutput)

In [ ]:
bestLambda = crossValidationOutput$lambda.min
confusionMatrix = predict(lassoFit, newx = t(exprs(dataCancerNoCancer)), type="class",s=bestLambda)
table(confusionMatrix, dataCancerNoCancer$disease)

ii) Write code to extract the number of false positives (non-cancer patients that are predicted to have cancer) and false negatives (cancer patients that are predicted to not have cancer).

## 3.  Elastic Net regression

The characteristic feature of Ridge regression is the penalty

$$\mbox{log}\left(L(\beta)\right) - \lambda \sum_i  \beta_i ^2,$$

while the penalty for Lasso regression is

$$\mbox{log}\left(L(\beta)\right) - \lambda \sum_i | \beta_i |.$$

In this Problem Set, we explore the penalty

$$\mbox{log}\left(L(\beta)\right) - \lambda \left(\alpha \sum_i | \beta_i | +  (1-\alpha) \sum_i  \beta_i ^2\right),$$

which is called __Elastic Net__.  

i) In parameter space, Ridge Regression corresponds to finding optimal parameters on a circle, while LASSO regression corresponds to finding optimal parameters on a diamond. What shape does Elastic Net correspond to?

ii). The `glmnet` package was built for Elastic Net regression. Look up the [glmnet package help files](https://www.rdocumentation.org/packages/glmnet/versions/3.0-2/topics/glmnet) to find out how to perform Elastic Net regression for a specific $\alpha$. Do this for $\alpha=0.5$, and return the confusion matrix.

iii) Perform a sweep over $\alpha=0$ to $\alpha=1$. Plot the number of species included versus $\alpha$.

Hint: The cv.glmnet output object has a returns a value `$nzero`, which is the number of nonzero factors, which is the number of species desired.  

iv) What $\alpha$ value minimizes the number of false positives (non-cancer patients that are predicted to have cancer)? What $\alpha$ value minimizes the number of false negatives (cancer patients that are predicted to not have cancer)?